# CellphoneRS based on Collaborative filtering

Import library

In [1]:
import pandas as pd
import numpy as np
import pickle as pkl

## Import data

In [2]:
phones: pd.DataFrame = pd.read_csv(
    filepath_or_buffer="data/20191226-items.csv",
    on_bad_lines="skip",
)

In [3]:
reviews: pd.DataFrame = pd.read_csv(
    filepath_or_buffer="data/20191226-reviews.csv",
    on_bad_lines="skip",
)

### Data cleaning

Create DataFrame users and write to csv file

In [4]:
# get the username column and drop duplicate usernames
usernames = reviews["name"].drop_duplicates().reset_index(drop=True)

# create user_id column with format ID1, ID2, ...
user_id_list = ["ID" + str(i + 1) for i in range(len(usernames))]

users = pd.DataFrame({"user_id": user_id_list, "username": usernames})

users.to_csv("data/user_info.csv", index=False)

In [5]:
print(" | ".join(list(users.columns)))
print(users.shape)
users.head()

user_id | username
(47323, 2)


,user_id,username
0,ID1,Janet
1,ID2,Luke Wyatt
2,ID3,Brooke
3,ID4,amy m. teague
4,ID5,tristazbimmer


Handle file `20191226-items.csv`

In [6]:
phones.head()

,asin,brand,title,url,image,rating,reviewUrl,totalReviews,price,originalPrice
0,B0000SX2UC,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.0
1,B0009N5L7K,Motorola,Motorola I265 phone,https://www.amazon.com/Motorola-i265-I265-phon...,https://m.media-amazon.com/images/I/419WBAVDAR...,3.0,https://www.amazon.com/product-reviews/B0009N5L7K,7,49.95,0.0
2,B000SKTZ0S,Motorola,MOTOROLA C168i AT&T CINGULAR PREPAID GOPHONE C...,https://www.amazon.com/MOTOROLA-C168i-CINGULAR...,https://m.media-amazon.com/images/I/71b+q3ydkI...,2.7,https://www.amazon.com/product-reviews/B000SKTZ0S,22,99.99,0.0
3,B001AO4OUC,Motorola,Motorola i335 Cell Phone Boost Mobile,https://www.amazon.com/Motorola-i335-Phone-Boo...,https://m.media-amazon.com/images/I/710UO8gdT+...,3.3,https://www.amazon.com/product-reviews/B001AO4OUC,21,0.00,0.0
4,B001DCJAJG,Motorola,Motorola V365 no contract cellular phone AT&T,https://www.amazon.com/Motorola-V365-contract-...,https://m.media-amazon.com/images/I/61LYNCVrrK...,3.1,https://www.amazon.com/product-reviews/B001DCJAJG,12,149.99,0.0


In [7]:
phones = phones[["asin", "brand", "title", "url", "image", "price", "originalPrice"]]

In [8]:
print(" | ".join(list(phones.columns)))
print(phones.shape)
phones.head()

asin | brand | title | url | image | price | originalPrice
(720, 7)


,asin,brand,title,url,image,price,originalPrice
0,B0000SX2UC,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,0.00,0.0
1,B0009N5L7K,Motorola,Motorola I265 phone,https://www.amazon.com/Motorola-i265-I265-phon...,https://m.media-amazon.com/images/I/419WBAVDAR...,49.95,0.0
2,B000SKTZ0S,Motorola,MOTOROLA C168i AT&T CINGULAR PREPAID GOPHONE C...,https://www.amazon.com/MOTOROLA-C168i-CINGULAR...,https://m.media-amazon.com/images/I/71b+q3ydkI...,99.99,0.0
3,B001AO4OUC,Motorola,Motorola i335 Cell Phone Boost Mobile,https://www.amazon.com/Motorola-i335-Phone-Boo...,https://m.media-amazon.com/images/I/710UO8gdT+...,0.00,0.0
4,B001DCJAJG,Motorola,Motorola V365 no contract cellular phone AT&T,https://www.amazon.com/Motorola-V365-contract-...,https://m.media-amazon.com/images/I/61LYNCVrrK...,149.99,0.0


Check image url

In [9]:
phones.iloc[237]['image']

'https://m.media-amazon.com/images/I/61ZeEiavaTL._AC_UY218_ML3_.jpg'

Handle file `20191226-reviews.csv`

In [10]:
reviews.head()

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


In [11]:
reviews = reviews[['asin','name','rating']]

Lets remane some wierd columns name

In [12]:
reviews.rename(
    columns={
        "name": "user_id",
        "rating": "rating",
    },
    inplace=True,
)

In [13]:
print(" | ".join(list(reviews.columns)))
print(reviews.shape)
reviews.head()

asin | user_id | rating
(67986, 3)


,asin,user_id,rating
0,B0000SX2UC,Janet,3
1,B0000SX2UC,Luke Wyatt,1
2,B0000SX2UC,Brooke,5
3,B0000SX2UC,amy m. teague,3
4,B0000SX2UC,tristazbimmer,4


### Create artifacts

In [14]:
reviews['user_id'].value_counts()

user_id
Amazon Customer        6507
Kindle Customer         360
Chris                    89
John                     87
Michael                  77
                       ... 
David L.G. Crawford       1
Paul Ramone               1
Anya Rochester            1
Glenda S.                 1
Owen Gonzalez             1
Name: count, Length: 47322, dtype: int64

In [15]:
reviews['user_id'].value_counts().shape

(47322,)

In [16]:
reviews['user_id'].unique().shape

(47323,)

In [17]:
# Lets store users who had at least rated more than 200 books
x = reviews['user_id'].value_counts() > 200

In [18]:
x[x].shape

(2,)

In [19]:
y = x[x].index

In [20]:
y

Index(['Amazon Customer', 'Kindle Customer'], dtype='object', name='user_id')

In [21]:
ratings = reviews[reviews['user_id'].isin(y)]

In [22]:
print(ratings.shape)
ratings.head()

(6867, 3)


,asin,user_id,rating
9,B0000SX2UC,Amazon Customer,3
27,B000SKTZ0S,Amazon Customer,1
45,B001AO4OUC,Amazon Customer,5
64,B001DCJAJG,Kindle Customer,4
109,B002AS9WEA,Amazon Customer,2


In [23]:
# Now join ratings with books
ratings_with_books = ratings.merge(phones, on='asin')

In [24]:
print(ratings_with_books.shape)
ratings_with_books.head()

(6867, 9)


,asin,user_id,rating,brand,title,url,image,price,originalPrice
0,B0000SX2UC,Amazon Customer,3,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,0.00,0.0
1,B000SKTZ0S,Amazon Customer,1,Motorola,MOTOROLA C168i AT&T CINGULAR PREPAID GOPHONE C...,https://www.amazon.com/MOTOROLA-C168i-CINGULAR...,https://m.media-amazon.com/images/I/71b+q3ydkI...,99.99,0.0
2,B001AO4OUC,Amazon Customer,5,Motorola,Motorola i335 Cell Phone Boost Mobile,https://www.amazon.com/Motorola-i335-Phone-Boo...,https://m.media-amazon.com/images/I/710UO8gdT+...,0.00,0.0
3,B001DCJAJG,Kindle Customer,4,Motorola,Motorola V365 no contract cellular phone AT&T,https://www.amazon.com/Motorola-V365-contract-...,https://m.media-amazon.com/images/I/61LYNCVrrK...,149.99,0.0
4,B002AS9WEA,Amazon Customer,2,Samsung,Samsung a167 Prepaid GoPhone (AT&T),https://www.amazon.com/Samsung-a167-Prepaid-Go...,https://m.media-amazon.com/images/I/61OXcZ-oef...,0.00,0.0


In [25]:
number_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [26]:
number_rating.head()

,title,rating
0,ASUS ZenFone 5Z (ZS620KL-S845-6G64G) - 6.2” FH...,10
1,ASUS ZenFone Max Plus ZB570TL-MT67-3G32G-BL - ...,15
2,Apple MGLW2LL/A iPad Air 2 9.7-Inch Retina Dis...,13
3,"Apple iPad Air MF529LL/A (32GB, Wi-Fi + at&T, ...",2
4,"Apple iPad mini 4 (32GB, Wi-Fi + Cellular, Spa...",2


In [27]:
number_rating.rename(columns={"rating": "num_of_rating"}, inplace=True)

In [28]:
number_rating.head()

,title,num_of_rating
0,ASUS ZenFone 5Z (ZS620KL-S845-6G64G) - 6.2” FH...,10
1,ASUS ZenFone Max Plus ZB570TL-MT67-3G32G-BL - ...,15
2,Apple MGLW2LL/A iPad Air 2 9.7-Inch Retina Dis...,13
3,"Apple iPad Air MF529LL/A (32GB, Wi-Fi + at&T, ...",2
4,"Apple iPad mini 4 (32GB, Wi-Fi + Cellular, Spa...",2


In [29]:
final_rating = ratings_with_books.merge(number_rating, on='title')

In [30]:
print(final_rating.shape)
final_rating.head()

(6867, 10)


,asin,user_id,rating,brand,title,url,image,price,originalPrice,num_of_rating
0,B0000SX2UC,Amazon Customer,3,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,0.00,0.0,1
1,B000SKTZ0S,Amazon Customer,1,Motorola,MOTOROLA C168i AT&T CINGULAR PREPAID GOPHONE C...,https://www.amazon.com/MOTOROLA-C168i-CINGULAR...,https://m.media-amazon.com/images/I/71b+q3ydkI...,99.99,0.0,1
2,B001AO4OUC,Amazon Customer,5,Motorola,Motorola i335 Cell Phone Boost Mobile,https://www.amazon.com/Motorola-i335-Phone-Boo...,https://m.media-amazon.com/images/I/710UO8gdT+...,0.00,0.0,1
3,B001DCJAJG,Kindle Customer,4,Motorola,Motorola V365 no contract cellular phone AT&T,https://www.amazon.com/Motorola-V365-contract-...,https://m.media-amazon.com/images/I/61LYNCVrrK...,149.99,0.0,1
4,B002AS9WEA,Amazon Customer,2,Samsung,Samsung a167 Prepaid GoPhone (AT&T),https://www.amazon.com/Samsung-a167-Prepaid-Go...,https://m.media-amazon.com/images/I/61OXcZ-oef...,0.00,0.0,2


In [31]:
# Lets take those books which got at least 50 rating of user
final_rating = final_rating[final_rating['num_of_rating'] >= 50]

In [32]:
final_rating.head()

,asin,user_id,rating,brand,title,url,image,price,originalPrice,num_of_rating
328,B00E6FGSHY,Kindle Customer,5,Samsung,"Samsung Galaxy S4, White Frost 16GB (AT&T)",https://www.amazon.com/Samsung-Galaxy-S4-White...,https://m.media-amazon.com/images/I/81suaO+v0m...,154.97,0.0,64
329,B00E6FGSHY,Amazon Customer,1,Samsung,"Samsung Galaxy S4, White Frost 16GB (AT&T)",https://www.amazon.com/Samsung-Galaxy-S4-White...,https://m.media-amazon.com/images/I/81suaO+v0m...,154.97,0.0,64
330,B00E6FGSHY,Amazon Customer,5,Samsung,"Samsung Galaxy S4, White Frost 16GB (AT&T)",https://www.amazon.com/Samsung-Galaxy-S4-White...,https://m.media-amazon.com/images/I/81suaO+v0m...,154.97,0.0,64
331,B00E6FGSHY,Amazon Customer,3,Samsung,"Samsung Galaxy S4, White Frost 16GB (AT&T)",https://www.amazon.com/Samsung-Galaxy-S4-White...,https://m.media-amazon.com/images/I/81suaO+v0m...,154.97,0.0,64
332,B00E6FGSHY,Amazon Customer,5,Samsung,"Samsung Galaxy S4, White Frost 16GB (AT&T)",https://www.amazon.com/Samsung-Galaxy-S4-White...,https://m.media-amazon.com/images/I/81suaO+v0m...,154.97,0.0,64


In [33]:
with open('artifacts/final_rating.pkl', 'wb') as file:
    pkl.dump(final_rating, file)

In [34]:
final_rating.shape

(1992, 10)

In [35]:
# lets drop the duplicates
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [36]:
final_rating.shape

(48, 10)

In [37]:
# Lets create a pivot table
user_phone_matrix = final_rating.pivot_table(columns='user_id', index='title', values= 'rating')

In [38]:
user_phone_matrix

user_id,Amazon Customer,Kindle Customer
title,,
"Motorola DROID Turbo XT1254, Black Ballistic Nylon 32GB (Verizon Wireless)",4.0,5.0
Motorola G6 – 32 GB – Unlocked (AT&T/Sprint/T-Mobile/Verizon) – Deep Indigo - (U.S. Warranty) - PAAE0011US,5.0,1.0
Nokia Lumia 635 8GB Unlocked GSM 4G LTE Windows 8.1 Quad-Core Phone - Black,5.0,5.0
"Samsung Galaxy A20 US Version Factory Unlocked Cell Phone with 32GB Memory, 6.4"" Screen, [SM-A205UZKAXAA], 12 Month Samsung US Warranty, GSM & CDMA Compatible, Black",5.0,5.0
"Samsung Galaxy J5 SM-J500H/DS GSM Factory Unlocked Smartphone, International Version (Gold)",2.0,2.0
Samsung Galaxy J7 - Verizon Carrier Locked No Contract Prepaid Smartphone,1.0,5.0
"Samsung Galaxy Note 3, Black 32GB (Verizon Wireless)",5.0,5.0
Samsung Galaxy Note 5 SM-N920V Gold 32GB (Verizon Wireless),1.0,5.0
"Samsung Galaxy Note 5, Black 64GB (Verizon Wireless)",1.0,5.0


In [39]:
user_phone_matrix.fillna(0, inplace=True)

In [40]:
print(user_phone_matrix.shape)
user_phone_matrix

(24, 2)


user_id,Amazon Customer,Kindle Customer
title,,
"Motorola DROID Turbo XT1254, Black Ballistic Nylon 32GB (Verizon Wireless)",4.0,5.0
Motorola G6 – 32 GB – Unlocked (AT&T/Sprint/T-Mobile/Verizon) – Deep Indigo - (U.S. Warranty) - PAAE0011US,5.0,1.0
Nokia Lumia 635 8GB Unlocked GSM 4G LTE Windows 8.1 Quad-Core Phone - Black,5.0,5.0
"Samsung Galaxy A20 US Version Factory Unlocked Cell Phone with 32GB Memory, 6.4"" Screen, [SM-A205UZKAXAA], 12 Month Samsung US Warranty, GSM & CDMA Compatible, Black",5.0,5.0
"Samsung Galaxy J5 SM-J500H/DS GSM Factory Unlocked Smartphone, International Version (Gold)",2.0,2.0
Samsung Galaxy J7 - Verizon Carrier Locked No Contract Prepaid Smartphone,1.0,5.0
"Samsung Galaxy Note 3, Black 32GB (Verizon Wireless)",5.0,5.0
Samsung Galaxy Note 5 SM-N920V Gold 32GB (Verizon Wireless),1.0,5.0
"Samsung Galaxy Note 5, Black 64GB (Verizon Wireless)",1.0,5.0


# Training model

In [41]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [42]:
book_sparse: csr_matrix = csr_matrix(user_phone_matrix)

Now import our clustering algorithm which is Nearest Neighbors this is an unsupervised machine learning algorithm

In [43]:
model: NearestNeighbors = NearestNeighbors(algorithm="brute")

In [44]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [45]:
distance, suggestion = model.kneighbors(user_phone_matrix.iloc[1,:].values.reshape(1,-1), n_neighbors=6 )

Let's say the `model` is your trained model

In [46]:
with open('artifacts/model.pkl', 'wb') as file:
    pkl.dump(model, file)

In [47]:
with open("artifacts/user_phone_matrix.pkl", "wb") as file:
    pkl.dump(user_phone_matrix, file)

In [48]:
distance

array([[0.        , 1.        , 1.        , 3.16227766, 3.60555128,
        4.        ]])

In [49]:
suggestion

array([[ 1, 12, 14,  4, 23,  3]], dtype=int64)

In [50]:
user_phone_matrix.iloc[1,:]

user_id
Amazon Customer    5.0
Kindle Customer    1.0
Name: Motorola G6 – 32 GB – Unlocked (AT&T/Sprint/T-Mobile/Verizon) – Deep Indigo - (U.S. Warranty) - PAAE0011US, dtype: float64

In [51]:
for i in range(len(suggestion)):
    print(user_phone_matrix.index[suggestion[i]])

Index(['Motorola G6 – 32 GB – Unlocked (AT&T/Sprint/T-Mobile/Verizon) – Deep Indigo - (U.S. Warranty) - PAAE0011US',
       'Samsung Galaxy S5 G900V Verizon 4G LTE Smartphone w/ 16MP Camera - Black - Verizon',
       'Samsung Galaxy S6 Edge, White Pearl 32GB (Verizon Wireless)',
       'Samsung Galaxy J5 SM-J500H/DS GSM Factory Unlocked Smartphone, International Version (Gold)',
       'Sony Xperia XA Ultra (F3213) 4G LTE Unlocked GSM Phone w/ 6” Borderless Display, 21.5MP+16MP Cameras, Octa-Core CPU - White',
       'Samsung Galaxy A20 US Version Factory Unlocked Cell Phone with 32GB Memory, 6.4" Screen, [SM-A205UZKAXAA], 12 Month Samsung US Warranty, GSM & CDMA Compatible, Black'],
      dtype='object', name='title')


In [52]:
user_phone_matrix.index[3]

'Samsung Galaxy A20 US Version Factory Unlocked Cell Phone with 32GB Memory, 6.4" Screen, [SM-A205UZKAXAA], 12 Month Samsung US Warranty, GSM & CDMA Compatible, Black'

In [53]:
#keeping phones name
phone_names = user_phone_matrix.index

In [54]:
with open('artifacts/phone_names.pkl', 'wb') as file:
    pkl.dump(phone_names, file)

In [55]:
phone_names[2]

'Nokia Lumia 635 8GB Unlocked GSM 4G LTE Windows 8.1 Quad-Core Phone - Black'

# Find image url

In [56]:
# final_rating['title'].value_counts()
ids = np.where(final_rating['title'] == "Motorola DROID Turbo XT1254, Black Ballistic Nylon 32GB (Verizon Wireless)")[0][0]

In [57]:
final_rating.iloc[ids]['image']

'https://m.media-amazon.com/images/I/81VMO0UpxfL._AC_UY218_ML3_.jpg'

In [58]:
phone_name_list = []
for phone_id in suggestion:
    phone_name_list.append(user_phone_matrix.index[phone_id])

In [59]:
phone_name_list[0]

Index(['Motorola G6 – 32 GB – Unlocked (AT&T/Sprint/T-Mobile/Verizon) – Deep Indigo - (U.S. Warranty) - PAAE0011US',
       'Samsung Galaxy S5 G900V Verizon 4G LTE Smartphone w/ 16MP Camera - Black - Verizon',
       'Samsung Galaxy S6 Edge, White Pearl 32GB (Verizon Wireless)',
       'Samsung Galaxy J5 SM-J500H/DS GSM Factory Unlocked Smartphone, International Version (Gold)',
       'Sony Xperia XA Ultra (F3213) 4G LTE Unlocked GSM Phone w/ 6” Borderless Display, 21.5MP+16MP Cameras, Octa-Core CPU - White',
       'Samsung Galaxy A20 US Version Factory Unlocked Cell Phone with 32GB Memory, 6.4" Screen, [SM-A205UZKAXAA], 12 Month Samsung US Warranty, GSM & CDMA Compatible, Black'],
      dtype='object', name='title')

In [60]:
ids_index = []
for name in phone_name_list[0]:
    ids = np.where(final_rating['title'] == name)[0][0]
    ids_index.append(ids)

In [61]:
for idx in ids_index:
    url = final_rating.iloc[idx]['image']
    print(url)

https://m.media-amazon.com/images/I/51L6DbMbvKL._AC_UY218_ML3_.jpg
https://m.media-amazon.com/images/I/91s1UjlYJHL._AC_UY218_ML3_.jpg
https://m.media-amazon.com/images/I/814WwhaRVuL._AC_UY218_ML3_.jpg
https://m.media-amazon.com/images/I/61alJun3JvL._AC_UY218_ML3_.jpg
https://m.media-amazon.com/images/I/81A-Ww8FqNL._AC_UY218_ML3_.jpg
https://m.media-amazon.com/images/I/51V-XG0uBDL._AC_UY218_ML3_.jpg


In [62]:
pkl.dump(model, open("artifacts/model.pkl", "wb"),)
pkl.dump(phone_names, open("artifacts/phone_names.pkl", "wb"),)
pkl.dump(final_rating, open("artifacts/final_rating.pkl", "wb"),)
pkl.dump(user_phone_matrix, open("artifacts/user_phone_matrix.pkl", "wb"),)

# Testing model

In [63]:
def recommend_book(phone_name):
    phone_id = np.where(user_phone_matrix.index == phone_name)[0][0]
    distance, suggestion = model.kneighbors(
        user_phone_matrix.iloc[phone_id, :].values.reshape(1, -1),
        n_neighbors=6,
    )

    for i in range(len(suggestion)):
        phones = user_phone_matrix.index[suggestion[i]]
        for j in phones:
            if j == phone_name:
                print(f"You searched '{phone_name}'\n")
                print("The suggestion books are: \n")
            else:
                print(j)

In [64]:
phone_name_list = "Motorola DROID Turbo XT1254, Black Ballistic Nylon 32GB (Verizon Wireless)"
recommend_book(phone_name_list)

You searched 'Motorola DROID Turbo XT1254, Black Ballistic Nylon 32GB (Verizon Wireless)'

The suggestion books are: 

Samsung Galaxy S8 Plus (S8+) (SM-G955FD) 4GB RAM / 64GB ROM 6.2-Inch 12MP 4G LTE Dual SIM FACTORY UNLOCKED - International Stock No Warranty (MIDNIGHT BLACK)
Samsung Galaxy A20 US Version Factory Unlocked Cell Phone with 32GB Memory, 6.4" Screen, [SM-A205UZKAXAA], 12 Month Samsung US Warranty, GSM & CDMA Compatible, Black
Samsung Galaxy S7 Edge G935FD 32GB Unlocked GSM 4G LTE
Nokia Lumia 635 8GB Unlocked GSM 4G LTE Windows 8.1 Quad-Core Phone - Black
Samsung Galaxy Note 3, Black 32GB (Verizon Wireless)
